## Step 2 - Database Engineering

In [1]:
# Importing Dependencies
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String,Numeric,Float
from sqlalchemy.orm import Session
import pandas as pd
Base = declarative_base()

In [2]:
# Importing the clean_hawaii_measurements.csv file as a dataframe
clean_hawaii_measurements_path = ('clean_hawaii_measurements.csv')
mes_df = pd.read_csv(clean_hawaii_measurements_path)

mes_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
# Importing the clean_hawaii_stations.csv file as a dataframe
clean_hawaii_stations_path = ("clean_hawaii_stations.csv")
sta_df = pd.read_csv(clean_hawaii_stations_path)
sta_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
# creating a "measurement" list of data to write to_dict() DataFrame metadata
measurement_data = mes_df.to_dict(orient='records')

In [5]:
# creating a "station" list of data to write to_dict() DataFrame metadata
station_data =sta_df.to_dict(orient='records')

In [6]:
# Creating db connection
engine = create_engine(f"sqlite:///hawaii.sqlite.db")

In [7]:
# Connecting to db
conn= engine.connect()


In [8]:
# Creating Measurement and Sation Classes

class Measurement(Base):
    __tablename__ = 'measurement'
    
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Integer)
    prcp = Column(String(255))
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__ = 'station'
    
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Integer)
    longitude = Column(Integer)
    elevation = Column(Integer)
    

In [9]:
# Creating both the Measurement and Station tables
Base.metadata.create_all(engine)

In [10]:
# Using MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Saving the reference to the `measurements` table as a variable called `measurement_table`
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)

In [12]:
# inserting the data into the table
conn.execute(measurement_table.insert(), measurement_data)

In [13]:
# fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', '0.08', 65),
 (2, 'USC00519397', '2010-01-02', '0.0', 63),
 (3, 'USC00519397', '2010-01-03', '0.0', 74),
 (4, 'USC00519397', '2010-01-04', '0.0', 76),
 (5, 'USC00519397', '2010-01-07', '0.06', 70)]

In [14]:
# Saving the reference to the `measurements` table as a variable called `measurement_table`
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [15]:
# inserting the data into the table
conn.execute(station_table.insert(), station_data)

In [16]:
# fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]